# ----------------------------------------
# IMPORTS GERAIS
# ----------------------------------------

In [17]:
import json
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import pickle
import mlflow
import mlflow.sklearn
from nltk.corpus import stopwords


pd.set_option('display.max_colwidth', None)


# ----------------------------------------
# CARREGAMENTO DOS DADOS
# ----------------------------------------

### Jobs

In [2]:
with open('data/jobs/jobs.json') as f:
    jobs = json.load(f)

jobs_rows = []
for job_id, data in jobs.items():
    info = data.get('informacoes_basicas', {})
    perfil = data.get('perfil_vaga', {})
    jobs_rows.append({
        'job_id': job_id,
        'titulo_vaga': info.get('titulo_vaga', ''),
        'vaga_sap': info.get('vaga_sap', 'Não'),
        'nivel_profissional_vaga': perfil.get('nivel profissional', ''),
        'nivel_ingles_vaga': perfil.get('nivel_ingles', ''),
        'nivel_espanhol_vaga': perfil.get('nivel_espanhol', ''),
        'areas_atuacao_vaga': perfil.get('areas_atuacao', ''),
        'descricao': perfil.get('principais_atividades', '') + ' ' + perfil.get('competencia_tecnicas_e_comportamentais', '')
    })
jobs_df = pd.DataFrame(jobs_rows)
print("Jobs shape:", jobs_df.shape)

Jobs shape: (14081, 8)


### Prospects

In [3]:
with open('data/prospects/prospects.json') as f:
    prospects = json.load(f)

prospects_rows = []
for job_id, data in prospects.items():
    for p in data.get('prospects', []):
        status = p.get('situacao_candidado', '').lower()
        label = 1 if 'contratado' in status or 'aprovado' in status else 0
        prospects_rows.append({
            'job_id': job_id,
            'applicant_id': p.get('codigo'),
            'situacao': p.get('situacao_candidado', ''),
            'label': label
        })
prospects_df = pd.DataFrame(prospects_rows)
print("Prospects shape:", prospects_df.shape)

Prospects shape: (53759, 4)


### Applicants

In [4]:
with open('data/applicants/applicants.json') as f:
    applicants = json.load(f)

applicants_rows = []
for app_id, data in applicants.items():
    form = data.get('formacao_e_idiomas', {})
    prof = data.get('informacoes_profissionais', {})
    applicants_rows.append({
        'applicant_id': app_id,
        'nivel_academico': form.get('nivel_academico', ''),
        'nivel_ingles': form.get('nivel_ingles', ''),
        'nivel_espanhol': form.get('nivel_espanhol', ''),
        'area_atuacao': prof.get('area_atuacao', ''),
        'cv_text': data.get('cv_pt', '')
    })
applicants_df = pd.DataFrame(applicants_rows)
print("Applicants shape:", applicants_df.shape)

Applicants shape: (42482, 6)


# ----------------------------------------
# ANÁLISE EXPLORATÓRIA (EDA)
# ----------------------------------------

In [5]:
print("\nJobs sample:\n", jobs_df.head())
print("\nProspects sample:\n", prospects_df.head())
print("\nApplicants sample:\n", applicants_df.head())

print("\nLabels balance:\n", prospects_df['label'].value_counts())

# Verificar nulos
print("\nMissing - Jobs:\n", jobs_df.isnull().sum())
print("\nMissing - Applicants:\n", applicants_df.isnull().sum())



Jobs sample:
   job_id                                        titulo_vaga vaga_sap  \
0   5185                                   Operation Lead -      Não   
1   5184                             Consultor PP/QM Sênior      Não   
2   5183                              ANALISTA PL/JR C/ SQL      Não   
3   5182                     Technical Architect - 11894809      Não   
4   5181  Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior      Não   

  nivel_profissional_vaga nivel_ingles_vaga nivel_espanhol_vaga  \
0                  Sênior          Avançado             Fluente   
1                  Sênior           Fluente              Nenhum   
2                Analista            Nenhum       Intermediário   
3                Analista            Básico              Básico   
4                  Sênior     Intermediário              Nenhum   

                  areas_atuacao_vaga  \
0       TI - Sistemas e Ferramentas-   
1  TI - Desenvolvimento/Programação-   
2       TI - Sistemas e Ferra

# ----------------------------------------
# FEATURE ENGINEERING - NLP match_score
# ----------------------------------------

In [6]:
# Merge de todas as tabelas
df = prospects_df.merge(jobs_df, on='job_id', how='inner').merge(applicants_df, on='applicant_id', how='inner')

print("\Merged data:\n", df.head())

# NLP TF-IDF

tfidf = TfidfVectorizer(max_features=1000)
df['descricao'] = df['descricao'].fillna('')
df['cv_text'] = df['cv_text'].fillna('')

job_desc = tfidf.fit_transform(df['descricao'])
cv_text = tfidf.transform(df['cv_text'])

# Similaridade do cosseno
df['match_score'] = [cosine_similarity(job_desc[i], cv_text[i])[0][0] for i in range(job_desc.shape[0])]

print("\match_score:\n", df[['match_score']].head())

os.makedirs("../notebooks/saved_models", exist_ok=True)
tfidf_path = "../notebooks/saved_models/tfidf_vectorizer.pkl"

with open(tfidf_path, "wb") as f:
    pickle.dump(tfidf, f)

\Merged data:
   job_id applicant_id                     situacao  label  \
0   4530        25632  Encaminhado ao Requisitante      0   
1   4530        25529  Encaminhado ao Requisitante      0   
2   4531        25364     Contratado pela Decision      1   
3   4531        25360  Encaminhado ao Requisitante      0   
4   4533        26338     Contratado pela Decision      1   

                                         titulo_vaga vaga_sap  \
0                                CONSULTOR CONTROL M      Não   
1                                CONSULTOR CONTROL M      Não   
2  2021-2607395-PeopleSoft Application Engine-Dom...      Não   
3  2021-2607395-PeopleSoft Application Engine-Dom...      Não   
4  2021-2605708-Microfocus Application Life Cycle...      Não   

  nivel_profissional_vaga nivel_ingles_vaga nivel_espanhol_vaga  \
0                   Pleno            Nenhum              Nenhum   
1                   Pleno            Nenhum              Nenhum   
2                  Sênior 

<>:4: SyntaxWarning: invalid escape sequence '\M'
<>:18: SyntaxWarning: invalid escape sequence '\m'
<>:4: SyntaxWarning: invalid escape sequence '\M'
<>:18: SyntaxWarning: invalid escape sequence '\m'
/tmp/ipykernel_38306/415529653.py:4: SyntaxWarning: invalid escape sequence '\M'
  print("\Merged data:\n", df.head())
/tmp/ipykernel_38306/415529653.py:18: SyntaxWarning: invalid escape sequence '\m'
  print("\match_score:\n", df[['match_score']].head())


\match_score:
    match_score
0     0.410945
1     0.186244
2     0.029640
3     0.051904
4     0.247603


In [7]:
df.to_csv('data/df_completo.csv', index=False)

colunas = [
    "nivel_profissional_vaga",
    "nivel_ingles_vaga",
    "nivel_ingles",
    "nivel_academico",
    "descricao",
    "cv_text"

]
df[colunas].to_csv("data/selected_df_columns.csv", index=False)

In [8]:
colunas = [
    "nivel_profissional_vaga",
    "nivel_ingles_vaga",
    "nivel_ingles",
    "nivel_academico",
    "descricao",
    "cv_text"
]


df_filtrado = df[df['label'] == 1][colunas]
df_filtrado.to_csv("data/selected_df_columns_label_1.csv", index=False)

In [21]:
print(df_filtrado[["nivel_profissional_vaga", "nivel_ingles_vaga", "nivel_ingles" ,"nivel_academico"]].iloc[0])

nivel_profissional_vaga                      Sênior
nivel_ingles_vaga                            Nenhum
nivel_ingles                               Avançado
nivel_academico            Ensino Superior Completo
Name: 2, dtype: object


# ----------------------------------------
# PRÉ-PROCESSAMENTO
# ----------------------------------------

In [9]:
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()
le4 = LabelEncoder()

df['nivel_profissional_vaga_enc'] = le1.fit_transform(df['nivel_profissional_vaga'].astype(str))
df['nivel_ingles_vaga_enc'] = le2.fit_transform(df['nivel_ingles_vaga'].astype(str))
df['nivel_ingles_enc'] = le3.fit_transform(df['nivel_ingles'].astype(str))
df['nivel_academico_enc'] = le4.fit_transform(df['nivel_academico'].astype(str))

features = [
    'match_score',
    'nivel_profissional_vaga_enc',
    'nivel_ingles_vaga_enc',
    'nivel_ingles_enc',
    'nivel_academico_enc'
]
X = df[features]
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42, test_size=0.2)


with open('saved_models/le1_nivel_profissional_vaga.pkl', 'wb') as f:
    pickle.dump(le1, f)
with open('saved_models/le2_nivel_ingles_vaga.pkl', 'wb') as f:
    pickle.dump(le2, f)
with open('saved_models/le3_nivel_ingles.pkl', 'wb') as f:
    pickle.dump(le3, f)
with open('saved_models/le4_nivel_academico.pkl', 'wb') as f:
    pickle.dump(le4, f)


colunas = [
    "nivel_profissional_vaga_enc",
    "nivel_ingles_vaga_enc",
    "nivel_ingles_enc",
    "nivel_academico_enc",
    "match_score"

]
df[colunas].to_csv("data/selected_features.csv", index=False)


# ----------------------------------------
# TREINAMENTO DO MODELO
# ----------------------------------------

In [10]:
mlflow.set_experiment("datathon_ml_pipeline")

with mlflow.start_run(run_name="RandomForest_MatchScore"):
    n_estimators = 100
    random_state = 42

    model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    print("Accuracy:", acc)
    print(classification_report(y_test, y_pred))

    # MLflow tracking
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("random_state", random_state)
    mlflow.log_metric("accuracy", acc)

    # Salvar o modelo dentro do MLflow
    mlflow.sklearn.log_model(
                                model, 
                                "random_forest_model", 
                                input_example=X_test.iloc[:2]
                            )

print("\n Modelo registrado no MLflow Tracking Server.")


2025/07/20 19:17:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Accuracy: 0.720687742651137
              precision    recall  f1-score   support

           0       0.84      0.82      0.83      7453
           1       0.21      0.23      0.22      1562

    accuracy                           0.72      9015
   macro avg       0.52      0.52      0.52      9015
weighted avg       0.73      0.72      0.72      9015



/home/alberone/projects/fiap/ml-engineering/ml-tech-fiap-datathon-decision/venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/home/alberone/projects/fiap/ml-engineering/ml-tech-fiap-datathon-decision/venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains


 Modelo registrado no MLflow Tracking Server.


# ----------------------------------------
# VALIDAÇÃO + EXPORTAÇÃO
# ----------------------------------------

In [11]:
y_pred = model.predict(X_test)
print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Salvar modelo com pickle
with open('saved_models/modelo_datathon.pkl', 'wb') as f:
    pickle.dump(model, f)

print("\nModelo salvo como modelo_datathon.pkl")

# Inferência
exemplo = X_test.iloc[0:1]
print("\Entrada para inferência:\n", exemplo)
prob = model.predict_proba(exemplo)
print("\nProbabilidade de contratação:", prob[0][1])

# DataFrame de features de treino
X_train.to_csv("../notebooks/saved_models/train_features.csv", index=False)

<>:13: SyntaxWarning: invalid escape sequence '\E'
<>:13: SyntaxWarning: invalid escape sequence '\E'
/tmp/ipykernel_38306/2134396194.py:13: SyntaxWarning: invalid escape sequence '\E'
  print("\Entrada para inferência:\n", exemplo)



Accuracy: 0.720687742651137

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.82      0.83      7453
           1       0.21      0.23      0.22      1562

    accuracy                           0.72      9015
   macro avg       0.52      0.52      0.52      9015
weighted avg       0.73      0.72      0.72      9015


Modelo salvo como modelo_datathon.pkl
\Entrada para inferência:
        match_score  nivel_profissional_vaga_enc  nivel_ingles_vaga_enc  \
22607     0.339023                            9                      4   

       nivel_ingles_enc  nivel_academico_enc  
22607                 0                    0  

Probabilidade de contratação: 0.0


# ----------------------------------------
# UTIL
# ----------------------------------------

In [12]:
import pickle

# Carregar o modelo treinado
with open('saved_models/modelo_datathon.pkl', 'rb') as f:
    model = pickle.load(f)

# Selecionar 200 amostras dos dados de treino
X_sample = X_train.iloc[:200]

# Fazer a inferência (predição)
y_pred = model.predict(X_sample)
y_prob = model.predict_proba(X_sample)[:, 1]  # Probabilidade da classe positiva

# Filtrar apenas os que têm grande probabilidade de contratação (> 0.8)
grande_prob_idx = y_prob > 0.8
X_grande_prob = X_sample[grande_prob_idx]
y_pred_grande_prob = y_pred[grande_prob_idx]
y_prob_grande_prob = y_prob[grande_prob_idx]

# Imprimir resultados
for i, idx in enumerate(X_grande_prob.index):
    print(f"Amostra {idx}:")
    print("Features:", X_grande_prob.loc[idx].values)
    print("Predição:", y_pred_grande_prob[i])
    print("Probabilidade de contratação:", y_prob_grande_prob[i])
    print("-" * 40)

Amostra 2470:
Features: [0.03876773 7.         3.         0.         0.        ]
Predição: 1
Probabilidade de contratação: 0.92
----------------------------------------
Amostra 38534:
Features: [0.25515319 0.         1.         0.         0.        ]
Predição: 1
Probabilidade de contratação: 0.81
----------------------------------------
Amostra 6591:
Features: [ 0.02681892 11.          4.          0.          0.        ]
Predição: 1
Probabilidade de contratação: 0.92
----------------------------------------
Amostra 12520:
Features: [0.01438726 9.         0.         0.         0.        ]
Predição: 1
Probabilidade de contratação: 0.83
----------------------------------------
Amostra 25813:
Features: [ 0.2249668 11.         3.         0.         0.       ]
Predição: 1
Probabilidade de contratação: 0.85
----------------------------------------
Amostra 33835:
Features: [0.64111128 9.         4.         0.         0.        ]
Predição: 1
Probabilidade de contratação: 0.93
------------------

In [13]:
import pandas as pd

resultados = X_grande_prob.copy()
resultados['predicao'] = y_pred_grande_prob
resultados['prob_contratacao'] = y_prob_grande_prob
print(resultados)

       match_score  nivel_profissional_vaga_enc  nivel_ingles_vaga_enc  \
2470      0.038768                            7                      3   
38534     0.255153                            0                      1   
6591      0.026819                           11                      4   
12520     0.014387                            9                      0   
25813     0.224967                           11                      3   
33835     0.641111                            9                      4   
13202     0.280536                            6                      2   
9055      0.202797                            7                      4   
9459      0.202797                            9                      4   

       nivel_ingles_enc  nivel_academico_enc  predicao  prob_contratacao  
2470                  0                    0         1          0.920000  
38534                 0                    0         1          0.810000  
6591                  0           